In [1]:
from zipline.pipeline.builtin import *

In [2]:
from zipline.pipeline import Pipeline
from zipline.research import run_pipeline
from zipline.pipeline.data import CNEquityPricing

# 分类器
返回值类型为整数、字符型对象类型，称为分类器。

In [3]:
def make_pipeline():
    s = Sector()
    return Pipeline(
        columns={
            '流通市值': CNEquityPricing.market_cap.latest,
            '行业': s,
        }, screen=s.eq(101))

In [4]:
run_pipeline(make_pipeline(), '2020-07-20', '2020-07-30')

流通市值   行业
datetime                  asset                           
2020-07-20 00:00:00+00:00 南 玻Ａ(000012)   1.063038e+10  101
                          深天地Ａ(000023)   2.363010e+09  101
                          中金岭南(000060)   1.617067e+10  101
                          冀东水泥(000401)   2.380573e+10  101
                          *ST藏格(000408)  3.073030e+09  101
...                                               ...  ...
2020-07-30 00:00:00+00:00 ST安泰(600408)   1.852510e+09  101
                          三友化工(600409)   1.300540e+10  101
                          ST柳化(600423)   2.244330e+09  101
                          青松建化(600425)   7.348950e+09  101
                          华鲁恒升(600426)   3.767266e+10  101

[5153 rows x 2 columns]

# 过滤器

## `指数`与`股票`

In [5]:
def make_pipeline():
    return Pipeline(
        columns={
            '收盘价': CNEquityPricing.close.latest,
        }, screen=IsIndex())
run_pipeline(make_pipeline(), '2020-07-10', '2020-07-30')

收盘价
datetime                  asset                   
2020-07-10 00:00:00+00:00 上证指数(1000001)   3450.594
                          Ａ股指数(1000002)   3616.870
                          Ｂ股指数(1000003)    244.906
                          工业指数(1000004)   2842.173
                          商业指数(1000005)   3635.511
...                                            ...
2020-07-30 00:00:00+00:00 信息安全(1399994)   2273.280
                          基建工程(1399995)   3732.097
                          智能家居(1399996)   3500.574
                          中证白酒(1399997)  11616.693
                          中证煤炭(1399998)   1177.014

[8265 rows x 1 columns]

In [6]:
def make_pipeline():
    return Pipeline(
        columns={
            '收盘价': CNEquityPricing.close.latest,
        }, screen=IsStock())
run_pipeline(make_pipeline(), '2020-07-10', '2020-07-30')

收盘价
datetime                  asset               
2020-07-10 00:00:00+00:00 平安银行(000001)   15.53
                          万 科Ａ(000002)   30.63
                          国农科技(000004)   32.83
                          世纪星源(000005)    2.89
                          深振业Ａ(000006)    7.81
...                                        ...
2020-07-30 00:00:00+00:00 三元股份(600429)    5.74
                          北方导航(600435)    8.90
                          片仔癀(600436)   197.07
                          通威股份(600438)   24.77
                          瑞贝卡(600439)     3.41

[56811 rows x 1 columns]

## `default_us_equity_universe_mask`
最低市值必须大于等于'minimum_market_cap'，默认为5亿。

In [7]:
def make_pipeline1():
    return Pipeline(
        columns={
            '流通市值': CNEquityPricing.market_cap.latest,
        })

In [8]:
result1 = run_pipeline(make_pipeline1(), '2020-07-30', '2020-07-30')
print('行数', result1.shape[0])
cond = result1['流通市值'] >= 500000000
actual = len(result1[cond])
print('流通市值 >= 500000000 行数', actual)

行数 3136
流通市值 >= 500000000 行数 2579


In [9]:
def make_pipeline2():
    return Pipeline(
        columns={
            '流通市值': CNEquityPricing.market_cap.latest,
        },
        screen=default_us_equity_universe_mask())


result2 = run_pipeline(make_pipeline2(), '2020-07-30', '2020-07-30')
assert len(result2) == actual

## `make_us_equity_universe`

In [10]:
from zipline.pipeline import factors, filters, classifiers

In [11]:
def make_pipeline():
    f = factors.AverageDollarVolume(
        window_length=200).downsample('month_start')
    return Pipeline(
        columns={
            'sector': Sector(),
            '成交额': f,
            '排位': f.top(30, groupby=Sector(), mask=default_us_equity_universe_mask()),
        }, screen=default_us_equity_universe_mask())

In [12]:
df = run_pipeline(make_pipeline(), '2020-07-30', '2020-07-30')
df

sector           成交额     排位
datetime                  asset                                    
2020-07-30 00:00:00+00:00 平安银行(000001)     103  1.409211e+09   True
                          万 科Ａ(000002)     104  2.027297e+09   True
                          国农科技(000004)     206  9.593482e+07  False
                          世纪星源(000005)     310  3.480987e+07  False
                          深振业Ａ(000006)     104  7.144401e+07  False
...                                        ...           ...    ...
                          三元股份(600429)     102  2.416808e+07  False
                          北方导航(600435)     310  1.294475e+08  False
                          片仔癀(600436)      206  5.167972e+08   True
                          通威股份(600438)     102  9.370172e+08   True
                          瑞贝卡(600439)      102  3.564076e+07  False

[2579 rows x 3 columns]

In [13]:
def make_pipeline():
    return Pipeline(
        columns={
            'universe': make_us_equity_universe(500),
        })
df = run_pipeline(make_pipeline(), '2020-07-30', '2020-07-30')
df

universe
datetime                  asset                  
2020-07-30 00:00:00+00:00 平安银行(000001)       True
                          万 科Ａ(000002)       True
                          国农科技(000004)      False
                          世纪星源(000005)      False
                          深振业Ａ(000006)       True
...                                           ...
                          信息安全(1399994)     False
                          基建工程(1399995)     False
                          智能家居(1399996)     False
                          中证白酒(1399997)     False
                          中证煤炭(1399998)     False

[3136 rows x 1 columns]

In [14]:
# 筛选出总体
df[df['universe']]

universe
datetime                  asset                 
2020-07-30 00:00:00+00:00 平安银行(000001)      True
                          万 科Ａ(000002)      True
                          深振业Ａ(000006)      True
                          全新好(000007)       True
                          中国宝安(000009)      True
...                                          ...
                          华胜天成(600410)      True
                          小商品城(600415)      True
                          江淮汽车(600418)      True
                          片仔癀(600436)       True
                          通威股份(600438)      True

[495 rows x 1 columns]

以下总体选择在实现中存在问题：
1. 部分行业可能数量不足；
2. 并未考虑行业最大比例参数；

## `Q500US`

In [16]:
def make_pipeline():
    return Pipeline(
        columns={
            '收盘价': CNEquityPricing.close.latest,
        }, screen=Q500US())
run_pipeline(make_pipeline(), '2020-07-30', '2020-07-30')

收盘价
datetime                  asset               
2020-07-30 00:00:00+00:00 平安银行(000001)   13.54
                          万 科Ａ(000002)   27.11
                          深振业Ａ(000006)    7.53
                          全新好(000007)     8.54
                          中国宝安(000009)    8.71
...                                        ...
                          华胜天成(600410)   14.59
                          小商品城(600415)    7.01
                          江淮汽车(600418)    9.77
                          片仔癀(600436)   197.07
                          通威股份(600438)   24.77

[495 rows x 1 columns]

## `Q1500US`

In [17]:
def make_pipeline():
    return Pipeline(
        columns={
            '收盘价': CNEquityPricing.close.latest,
        }, screen=Q1500US())
run_pipeline(make_pipeline(), '2020-07-30', '2020-07-30')

收盘价
datetime                  asset               
2020-07-30 00:00:00+00:00 平安银行(000001)   13.54
                          万 科Ａ(000002)   27.11
                          深振业Ａ(000006)    7.53
                          全新好(000007)     8.54
                          中国宝安(000009)    8.71
...                                        ...
                          三元股份(600429)    5.74
                          北方导航(600435)    8.90
                          片仔癀(600436)   197.07
                          通威股份(600438)   24.77
                          瑞贝卡(600439)     3.41

[1156 rows x 1 columns]

## `Q3000US`

In [18]:
def make_pipeline():
    return Pipeline(
        columns={
            '收盘价': CNEquityPricing.close.latest,
        }, screen=Q3000US())


run_pipeline(make_pipeline(), '2020-07-30', '2020-07-30')

收盘价
datetime                  asset               
2020-07-30 00:00:00+00:00 平安银行(000001)   13.54
                          万 科Ａ(000002)   27.11
                          国农科技(000004)   29.11
                          深振业Ａ(000006)    7.53
                          全新好(000007)     8.54
...                                        ...
                          三元股份(600429)    5.74
                          北方导航(600435)    8.90
                          片仔癀(600436)   197.07
                          通威股份(600438)   24.77
                          瑞贝卡(600439)     3.41

[1797 rows x 1 columns]